In [1]:
import spotipy
import spotipy.util as util
import webbrowser
import json
import urllib.request
from nltk.corpus import stopwords
from nltk import pos_tag
import requests
import collections

In [5]:
# key_file is a variable representing my key txt file
# the 'r' file 
with open("news_key.txt", 'r') as key_file:
    news_key = key_file.read()
# close the file
key_file.close()
print(news_key)

7bd1ea7f9c154f85a63cb73489aa246c


In [7]:
country = "us"
category = "technology"
url = f"https://newsapi.org/v2/top-headlines?country={country}&category={category}&apiKey={news_key}"
print(url)

https://newsapi.org/v2/top-headlines?country=us&category=technology&apiKey=7bd1ea7f9c154f85a63cb73489aa246c


In [8]:
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)

In [9]:
news_articles = json.loads(response.read())

In [10]:
news_articles

{'status': 'ok',
 'totalResults': 31,
 'articles': [{'source': {'id': None, 'name': '9to5Mac'},
   'author': 'Filipe Espósito',
   'title': 'Apple announces Vision Pro international expansion in two more countries - 9to5Mac',
   'description': 'Following an exclusive launch in the US in February, Apple began the international expansion of Vision Pro in June. Now...',
   'url': 'https://9to5mac.com/2024/10/30/apple-vision-pro-international-expansion/',
   'urlToImage': 'https://i0.wp.com/9to5mac.com/wp-content/uploads/sites/6/2024/10/vision-pro-eyesight-2.jpeg?resize=1200%2C628&quality=82&strip=all&ssl=1',
   'publishedAt': '2024-10-31T03:14:00Z',
   'content': 'Following an exclusive launch in the US in February, Apple began the international expansion of Vision Pro in June. Now the company is ready to bring Apple Vision Pro to even more countries, as the c… [+1321 chars]'},
  {'source': {'id': None, 'name': 'KGW.com'},
   'author': 'Katherine Cook',
   'title': "Portland radio DJ surv